In [2]:
!pip install mercantile

In [6]:
import os
import requests
import mercantile
from shapely.geometry import shape, Polygon
from concurrent.futures import ThreadPoolExecutor, as_completed
from time import sleep
import os
import folium
import threading
from http.server import HTTPServer, SimpleHTTPRequestHandler
from IPython.display import display

In [4]:


# Helper function to download a single tile with retries
def download_tile(xyz_url, z, x, y, xyz_dir, retries=3, delay=1):
    tile_url = xyz_url.format(z=z, x=x, y=y)
    tile_path = os.path.join(xyz_dir, str(z), str(x), f"{y}.png")
    
    # Create directories if they do not exist
    os.makedirs(os.path.dirname(tile_path), exist_ok=True)

    # Check if tile already exists
    if os.path.exists(tile_path):
        return  # Tile already cached
    
    # Try to download the tile with retries
    attempt = 0
    while attempt < retries:
        try:
            response = requests.get(tile_url, timeout=10)  # Set a timeout for requests
            if response.status_code == 200:
                with open(tile_path, 'wb') as f:
                    f.write(response.content)
                return  # Successfully downloaded
            else:
                print(f"Failed to download tile {z}/{x}/{y}, status: {response.status_code}")
                break  # Don't retry if server gives a bad response
        except Exception as e:
            print(f"Error downloading tile {z}/{x}/{y}: {e}")
            attempt += 1
            if attempt < retries:
                print(f"Retrying tile {z}/{x}/{y}... (attempt {attempt}/{retries})")
                sleep(delay)  # Wait before retrying
    print(f"Failed to download tile {z}/{x}/{y} after {retries} attempts")

# Main function to download and cache XYZ tiles
def cache_xyz_tiles(xyz_url, xyz_dir, zoom, aoi=None, max_workers=10, verbose=True):
    """
    Downloads XYZ tiles and caches them locally.

    Args:
    - xyz_url (str): The URL template for the XYZ tile service.
    - xyz_dir (str): Directory to store the cached tiles.
    - zoom (int or range): The zoom level(s) for which to download tiles.
    - aoi (dict, optional): A GeoJSON object defining the Area of Interest (AOI). Defaults to global.
    - max_workers (int, optional): Number of threads for concurrent downloads. Default is 10.
    - verbose (bool, optional): If True, prints progress information. Default is True.
    """
    
    # If no AOI is provided, use the entire world extent
    if aoi is None:
        # Global extent (Web Mercator bounds)
        aoi_extent = (-180, -85.0511, 180, 85.0511)  # As a tuple (min_lon, min_lat, max_lon, max_lat)
    else:
        # Extract bounding box from AOI GeoJSON
        polygon = shape(aoi['features'][0]['geometry'])
        if isinstance(polygon, Polygon):
            aoi_extent = polygon.bounds  # This will return (min_lon, min_lat, max_lon, max_lat)
        else:
            raise ValueError("AOI geometry must be a Polygon")

    min_lon, min_lat, max_lon, max_lat = aoi_extent
    
    # Ensure zoom is iterable (even if it's just one level)
    if isinstance(zoom, int):
        zoom_levels = [zoom]
    else:
        zoom_levels = range(zoom[0], zoom[1] + 1)
    
    # Use mercantile to calculate the list of tiles
    total_tiles = 0
    for z in zoom_levels:
        tile_list = list(mercantile.tiles(min_lon, min_lat, max_lon, max_lat, zooms=[z]))
        total_tiles += len(tile_list)
        if verbose:
            print(f"Zoom level {z}: {len(tile_list)} tiles to download.")

        # Download the tiles using multithreading
        futures = []
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            for tile in tile_list:
                futures.append(executor.submit(download_tile, xyz_url, tile.z, tile.x, tile.y, xyz_dir))

            # Track progress and handle exceptions
            tile_count = 0
            for future in as_completed(futures):
                try:
                    future.result()  # Raise any exceptions caught by threads
                except Exception as e:
                    print(f"Error in future: {e}")

                tile_count += 1
                if verbose:
                    print(f"Progress: {tile_count}/{len(tile_list)} tiles downloaded for zoom {z}", end="\r")
    
    print("\nTile caching complete.")




# Example usage:

In [5]:
# Example usage:
# Use the global default extent without providing AOI
xyz_url = "https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/cca86b15cd17eebfec8b381ec7166df4-f607df06b0198e13a0b51c3376cff8c1/tiles/{z}/{x}/{y}.png"
xyz_dir = "/Users/maples/GitHub/EarthEngine_2_Globe/caches"
zoom = range(0,4,1)  # Download tiles for zoom level 4 globally

# Call the function without an AOI
cache_xyz_tiles(xyz_url, xyz_dir, zoom)

Zoom level 0: 1 tiles to download.
Failed to download tile 0/0/0, status: 400
Failed to download tile 0/0/0 after 3 attempts
Zoom level 1: 4 tiles to download. zoom 0
Failed to download tile 1/1/1, status: 400
Failed to download tile 1/1/1 after 3 attempts
Progress: 4/4 tiles downloaded for zoom 1
Tile caching complete.


In [7]:


# Step 1: Serve XYZ tiles from the local directory
def serve_xyz_tiles(directory, port=8000):
    """
    Serve the XYZ tiles from a local directory using a simple HTTP server.
    
    Args:
    - directory (str): Path to the directory containing the XYZ tiles.
    - port (int): Port to run the local server. Defaults to 8000.
    """
    os.chdir(directory)
    handler = SimpleHTTPRequestHandler
    httpd = HTTPServer(('localhost', port), handler)
    print(f"Serving XYZ tiles at http://localhost:{port}")
    httpd.serve_forever()

# Step 2: Display the tiles on a Folium map (inline in a Jupyter notebook)
def display_tiles_on_folium_inline(directory, port=8000):
    """
    Display the XYZ tiles from a local directory on a Folium map inline in a Jupyter notebook.
    
    Args:
    - directory (str): Path to the directory containing the XYZ tiles.
    - port (int): Port to run the local server. Defaults to 8000.
    """
    # Start the local tile server in a separate thread
    server_thread = threading.Thread(target=serve_xyz_tiles, args=(directory, port))
    server_thread.daemon = True
    server_thread.start()

    # Create the Folium map
    m = folium.Map(location=[0, 0], zoom_start=2)

    # Add the custom tile layer pointing to the local tile server
    tile_url = f"http://localhost:{port}/{{z}}/{{x}}/{{y}}.png"
    folium.TileLayer(tile_url, attr="Local XYZ Tiles", name="Local Tiles").add_to(m)

    # Add layer control
    folium.LayerControl().add_to(m)

    # Display the map inline
    display(m)  # Display the map in the notebook

# Example usage:
directory = "/Users/maples/GitHub/EarthEngine_2_Globe/caches"  # Replace with the path to your XYZ tile cache
display_tiles_on_folium_inline(directory)


Serving XYZ tiles at http://localhost:8000


127.0.0.1 - - [04/Oct/2024 15:18:42] code 404, message File not found
127.0.0.1 - - [04/Oct/2024 15:18:42] "GET /2/1/1.png HTTP/1.1" 404 -
127.0.0.1 - - [04/Oct/2024 15:18:42] code 404, message File not found
127.0.0.1 - - [04/Oct/2024 15:18:42] "GET /2/2/1.png HTTP/1.1" 404 -
127.0.0.1 - - [04/Oct/2024 15:18:42] code 404, message File not found
127.0.0.1 - - [04/Oct/2024 15:18:42] "GET /2/1/2.png HTTP/1.1" 404 -
127.0.0.1 - - [04/Oct/2024 15:18:42] code 404, message File not found
127.0.0.1 - - [04/Oct/2024 15:18:42] "GET /2/2/2.png HTTP/1.1" 404 -
127.0.0.1 - - [04/Oct/2024 15:18:42] code 404, message File not found
127.0.0.1 - - [04/Oct/2024 15:18:42] "GET /2/0/3.png HTTP/1.1" 404 -
127.0.0.1 - - [04/Oct/2024 15:18:42] code 404, message File not found
127.0.0.1 - - [04/Oct/2024 15:18:42] "GET /2/3/3.png HTTP/1.1" 404 -
127.0.0.1 - - [04/Oct/2024 15:18:42] code 404, message File not found
127.0.0.1 - - [04/Oct/2024 15:18:42] "GET /2/3/2.png HTTP/1.1" 404 -
127.0.0.1 - - [04/Oct/2024 